In [38]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

import pandas as pd

print("I'm actually an old v2 python pretending to be a young, hip v3 python")

I'm actually an old v2 python pretending to be a young, hip v3 python


In [39]:
pd.__version__

u'0.24.2'

# 0 Check validity of Sentiment Analysis method 
### (using original dataset provided with paper)

The first set of cells is working with the training+test set data provided in the paper. I am loading in the json files, cleaning the comment field to remove return characters ('\n') and other things that cause problems when fed into the SentiTool. Then I am outputting a simple tab-delimited text file with the ID and comment field.

In [8]:
inpFiles = ["Bug_tt.json","Feature_tt.json","Rating_tt.json","UserExperience_tt.json"]
delimiter = "\t".encode('utf-8')

for entry in inpFiles:
    tempDF = pd.read_json(entry)
    tempDF["comment"]=tempDF["comment"].str.replace("\n"," ",regex=False)
    tempDF["comment"]=tempDF["comment"].str.replace("\r"," ",regex=False)
    tempDF["comment"]=tempDF["comment"].str.replace("\t"," ",regex=False)
    tempDF["comment"]=tempDF["comment"].str.rstrip() #white space at the end of a comment was creating a new line upon export
    tempDF["comment"]=tempDF["comment"].str.lstrip() #no reason to keep leading whitespaces, so stripping off
    tempDF[["id","comment"]].to_csv((entry.split(".")[0]+"_for_senti.txt"), header=True, index=False, sep=delimiter, encoding = "utf-8")
    print(entry,"processed")



Bug_tt.json processed
Feature_tt.json processed
Rating_tt.json processed
UserExperience_tt.json processed


### 0-1  Next use the standalone SentiStrength application

(This step must be performed manually outside of the notebook)

Use SentiStrength 2.3 from (http://sentistrength.wlv.ac.uk/)
Use Sept 21, 2011 configuration files downloaded from same site
Use the following settings
![settings0](sshot0-senti_settings0.png)
![settings1](sshot1-senti_settings1.png)
![settings1](sshot2-senti_settings2.png)
![settings1](sshot3-senti_settings3.png)
Use Sentiment Strength Analysis -> Analyze ALL Texts in File....As Above for ALL files in folder

Yes to echo the header in the results

Column 2 contains the text (do not use the default value of 3)

results will be saved with "+results" appended to the filename in the folder where the input files were stored.

#### 0-2 Now read the results back in and compare

The following reads both the original files and the results in as dataframes, merges them and compares the three categories of sentiment scores contained in the original _tt.json files with those that we just calculated. 

In [37]:
delimiter = "\t".encode('utf-8')

def find_max_sent(vect):
    pos=vect[0]
    neg=vect[1]
    if abs(pos)>abs(neg):
        return pos
    else:
        return neg
#***In cases of ties, the authors appear to have chosen to use the negative sentiment score instead of a neutral 0!

for entry in inpFiles:
    print("Loading", entry.split(".")[0])
    origDF = pd.read_json(entry)
    origDF["comment"]=origDF["comment"].str.replace("\n"," ",regex=False)
    origDF["comment"]=origDF["comment"].str.replace("\r"," ",regex=False)
    origDF["comment"]=origDF["comment"].str.replace("\t"," ",regex=False)
    origDF["comment"]=origDF["comment"].str.rstrip() #white space at the end of a comment was creating a new line upon export
    origDF["comment"]=origDF["comment"].str.lstrip() #no reason to keep leading whitespaces, so stripping off
    resultsDF = pd.read_csv(entry.split(".")[0]+"_for_senti+results.txt", sep=delimiter, encoding="utf-8")
    origDF = origDF.merge(resultsDF, on=["id","comment"])
    origDF["Score"] = origDF[["Positive","Negative"]].apply(find_max_sent, axis=1)
    n = origDF["id"].count()
    totScoreMatch = sum(origDF["sentiScore"]==origDF["Score"])
    negScoreMatch = sum(origDF["sentiScore_neg"]==origDF["Negative"])
    posScoreMatch = sum(origDF["sentiScore_pos"]==origDF["Positive"])
    
    print("SentiScore Matches:",totScoreMatch,"/",n," = ", totScoreMatch/n)
    print("PositiveSentiScore Matches:",posScoreMatch,"/",n," = ", posScoreMatch/n)
    print("NegativeSentiScore Matches:",negScoreMatch,"/",n," = ", negScoreMatch/n)

Loading Bug_tt
SentiScore Matches: 594 / 722  =  0.8227146814404432
PositiveSentiScore Matches: 650 / 722  =  0.9002770083102493
NegativeSentiScore Matches: 606 / 722  =  0.8393351800554016
Loading Feature_tt
SentiScore Matches: 487 / 597  =  0.8157453936348409
PositiveSentiScore Matches: 537 / 597  =  0.8994974874371859
NegativeSentiScore Matches: 502 / 597  =  0.8408710217755444
Loading Rating_tt
SentiScore Matches: 605 / 721  =  0.8391123439667129
PositiveSentiScore Matches: 658 / 721  =  0.912621359223301
NegativeSentiScore Matches: 616 / 721  =  0.8543689320388349
Loading UserExperience_tt
SentiScore Matches: 608 / 728  =  0.8351648351648352
PositiveSentiScore Matches: 650 / 728  =  0.8928571428571429
NegativeSentiScore Matches: 636 / 728  =  0.8736263736263736



***

# 1 Prepare sampled dataset for sentiment analysis

### This step will likely be unnecessary in the combined workflow notebook as we will instead already have a dataframe in memory

In [46]:
filename = "sample_384_all_coded.csv"
delimiter = "\t".encode('utf-8')


tempDF = pd.read_csv(filename, encoding="utf-8")
tempDF["processed_text"]=tempDF["processed_text"].str.replace("\n"," ",regex=False)
tempDF["processed_text"]=tempDF["processed_text"].str.replace("\r"," ",regex=False)
tempDF["processed_text"]=tempDF["processed_text"].str.replace("\t"," ",regex=False)
tempDF["processed_text"]=tempDF["processed_text"].str.rstrip() #white space at the end of a processed_text was creating a new line upon export
tempDF["processed_text"]=tempDF["processed_text"].str.lstrip() #no reason to keep leading whitespaces, so stripping off
tempDF[["id","processed_text"]].to_csv(filename+"_for_senti.txt", header=True, index=False, sep=delimiter, encoding = "utf-8")
print(filename+"_for_senti.txt saved for input to sentiStrength")



sample_384_all_coded.csv_for_senti.txt saved for input to sentiStrength



**See Step 0-1 for the correct configuration to use in the stand alone SentiStrength application**

(replace this cell with that information if we are removing all of step 0 when merging this notebook to the combined workflow document)



In [47]:
def find_max_sent(vect):
    pos=vect[0]
    neg=vect[1]
    if abs(pos)>abs(neg):
        return pos
    else:
        return neg #The authors of the paper appear to have used the negative score in the event of a tie

resultsDF = pd.read_csv(filename+"_for_senti+results.txt", sep=delimiter, encoding="utf-8")
tempDF = tempDF.merge(resultsDF, on=["id","processed_text"]) #could do it just on ID. Don't really need to bring processed text back in either

#assign a single sentiment score based on whichever value is further from neutral (0)
tempDF["sentiScore"] = tempDF[["Positive","Negative"]].apply(find_max_sent, axis=1)
#rename the other two sentiment score columns to match what they are called in the paper's original dataset
tempDF.rename(columns={"Positive":"sentiScore_pos","Negative":"sentiScore_neg"}, inplace=True)

tempDF.head(5)

,id,appTitle,userName,date,score,text,fileDate,fileCategories,contentRating,appId,reviewId,processed_text,label_UE,label_BR,label_FR,label_R,sentiScore_pos,sentiScore_neg,sentiScore
0,804084,PBS KIDS Video,A Google User,15-Aug-17,5,"It,s great gots lot of show it,s great 5 stars",2019-04-14,EDUCATION,Everyone,org.pbskids.video,1333153,"It's great, got lots of shows, 5 stars",0,0,0,1,3,-1,3
1,869576,IRS2Go,A Google User,23-Mar-15,5,Teacher Used the app to check on the status of...,2019-03-24,FINANCE,Everyone,gov.irs,1436222,Teacher Used the app to check on the status of...,1,0,0,0,1,-1,-1
2,425433,Google Play Games,NaN,19-Mar-19,1,It was enjoyable and educative a good one.,2019-03-24,ENTERTAINMENT,Teen,com.google.android.play.games,726513,It was enjoyable and educative a good one.,0,0,0,1,3,-1,3
3,855228,Venmo Mobile Wallet: Send & Receive Money,Linda Logan,05-Mar-19,5,great way to send or receive money,2019-03-08,FINANCE,Everyone,com.venmo,1414907,great way to send or receive money,0,0,0,1,3,-1,3
4,1010397,Love Poly - New puzzle game,Valerie P,10-Mar-19,2,the amount of ads is ridiculous.,2019-04-01,FAMILY,Everyone,com.love.poly.puzzle.game,1653018,the amount of ads is ridiculous.,0,0,0,1,1,-3,-3


In [50]:
#quick groupby analysis to see how many records exist for each sentiment score

tempDF.groupby(["sentiScore"]).appTitle.count()

sentiScore
-4     22
-3     20
-2     45
-1     79
 2     71
 3    121
 4     14
Name: appTitle, dtype: int64